In [1]:
import os
import os.path as path

import random
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import networkx as nx
from cdlib import algorithms

In [2]:
# fix seed (since the algorithms we used to label nodes are randomized)
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

## Load datasets

In [3]:
# dataset
DATASET = 'uselections'  # other datasets: "brexit", "abortion"
DATASETS_PATH = path.join(path.pardir, "datasets", "balanced_exposure")
DATASET_PATH = path.join(DATASETS_PATH, DATASET)

In [4]:
# load probabilities
PROBS_PATH = path.join(DATASET_PATH, f"{DATASET}_network_heterogeneous.txt")
df = pd.read_csv(PROBS_PATH, sep='\t', names=['User', 'Follower', 'Pa', 'Pb'])

In [5]:
# load seeds
A = [l.strip() for l in open(path.join(DATASET_PATH, "side1_seeds.txt")).readlines()]  # A = pro Hillary
B = [l.strip() for l in open(path.join(DATASET_PATH, "side2_seeds.txt")).readlines()]  # B = pro Trump

In [6]:
# remove outlier seed
A.remove('jjack9400')
df = df[(df.User != 'jjack9400') | (df.Follower != 'jjack9400')]

In [7]:
# label group attribute for user and follower
A1, B1 = df.User.isin(A), df.User.isin(B)
df['Group1'] = np.where(~A1 & ~B1, 'NA', np.where(A1, 'A', 'B')) 

A2, B2 = df.Follower.isin(A), df.Follower.isin(B)
df['Group2'] = np.where(~A2 & ~B2, 'NA', np.where(A2, 'A', 'B')) 

In [8]:
print('A:', len(set(A)) == len(A), len(A), ', B:', len(set(B)) == len(B), len(B))

A: True 84 , B: True 99


In [9]:
# find biggest connected component
G = nx.from_pandas_edgelist(df, 'User', 'Follower')
biggest_component = next(nx.algorithms.connected_components(G))

# discard the rest
G = G.subgraph(biggest_component)
df = df[df.User.isin(biggest_component) | df.Follower.isin(biggest_component)]
A = [a for a in A if a in biggest_component]
B = [b for b in B if b in biggest_component]
len(A), len(B)

(84, 99)

In [ ]:
# expand seeds to larger communities, hoping to find edges between A and B...
commA = algorithms.lemon(G, A, min_com_size=100, max_com_size=700)

# label group of new nodes
newdf = df.copy()
newdf.loc[newdf.User.isin(commA.communities[0]), 'Group1'] = 'A'
newdf.loc[newdf.Follower.isin(commA.communities[0]), 'Group2'] = 'A'

print(len(commA.communities[0]), len(set(commA.communities[0]).difference(set(A))))

In [ ]:
commB = algorithms.lemon(G, B, min_com_size=50, max_com_size=200)

newdf = df.copy()
newdf.loc[newdf.User.isin(commB.communities[0]), 'Group1'] = 'B'
newdf.loc[newdf.Follower.isin(commB.communities[0]), 'Group2'] = 'B'

print(len(commB.communities[0]), len(set(commB.communities[0]).difference(set(B))))

In [14]:
# intersections of thw two communities (should be empty)
assert len(set(commB.communities[0]).intersection(set(commA.communities[0]))) == 0

In [15]:
# are there any edges between A and B now?
newdf[(newdf.Group1 != 'NA') & (newdf.Group2 != 'NA')]

,User,Follower,Pa,Pb,Group1,Group2
2058,1492cstob,tommyshida,0.000968,0.018808,B,B
2059,1492cstob,trumpwallnow,0.001219,0.028179,B,B
2060,1492cstob,the_trump_train,0.000071,0.036391,B,B
2918,1_hoof_hearted,eldoncloudtrump,0.006747,0.086667,B,B
2924,1_hoof_hearted,thefoundingson,0.000937,0.008457,B,B
...,...,...,...,...,...,...
844807,healthandcents,gdltothepdx,0.001108,0.003908,B,B
844828,healthandcents,rob_mcwaters,0.000865,0.020705,B,B
844838,healthandcents,ejhirschberger,0.000226,0.005506,B,B
844866,healthandcents,drmartyfox,0.001571,0.011651,B,B


In [16]:
labeled = newdf[(newdf.Group1 != 'NA') | (newdf.Group2 != 'NA')]
newA = list(set(labeled[labeled.Group1 == 'A'].User.tolist() + labeled[labeled.Group2 == 'A'].Follower.tolist()))
newB = list(set(labeled[labeled.Group1 == 'B'].User.tolist() + labeled[labeled.Group2 == 'B'].Follower.tolist()))
newNA = list(set(labeled[labeled.Group1 == 'NA'].User.tolist() + labeled[labeled.Group2 == 'NA'].Follower.tolist()))
G2 = nx.from_pandas_edgelist(labeled, 'User', 'Follower')
biggest_component2 = next(nx.algorithms.connected_components(G2))

# save biggest component of new graph
G2 = G2.subgraph(biggest_component)
newdf = newdf[newdf.User.isin(biggest_component) & newdf.Follower.isin(biggest_component)]

In [32]:
newdf = newdf[(newdf.Group1 != 'NA') & (newdf.Group2 != 'NA')]

In [33]:
newdf.to_csv('new_graph.csv', index=False)